In [1]:
import numpy as np
import pandas as pd
import h5py
import gc

import sklearn as sk

from sklearn.model_selection import ParameterGrid




import tensorflow as tf
from tensorflow.keras import mixed_precision
from tensorflow.keras.utils import GeneratorEnqueuer

import os
base_dir = '/media/tord/T7/Thesis_ssd/MasterThesis3'
os.chdir(base_dir)

from Classes.Modeling.DynamicModels import DynamicModels
from Classes.Modeling.StaticModels import StaticModels
from Classes.DataProcessing.LoadData import LoadData
from Classes.DataProcessing.HelperFunctions import HelperFunctions
from Classes.DataProcessing.DataHandler import DataHandler
from Classes.DataProcessing.RamLoader import RamLoader
from Classes.Modeling.GridSearchResultProcessor import GridSearchResultProcessor
from Classes.DataProcessing.RamGen import get_batch



import sys


import random
import pprint
import re
import json


INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: GeForce RTX 3090, compute capability 8.6


In [2]:
load_args = {
    'earth_explo_only' : True,
    'noise_earth_only' : False,
    'noise_not_noise' : False,
    'downsample' : True,
    'upsample' : True,
    'frac_diff' : 1,
    'seed' : 1,
    'subsample_size' : 0.05,
    'balance_non_train_set' : True,
    'use_true_test_set' : False,
    'even_balance' : True
}
loadData = LoadData(**load_args)
train_ds, val_ds, test_ds = loadData.get_datasets()
noise_ds = loadData.noise_ds
handler = DataHandler(loadData)
helper = HelperFunctions()

2 2
2 2
2 2


Mapping train redundancy: : [--------------------------------------->] 100 %



Mapping validation redundancy: : [--------------------------------------->] 100 %

Mapping test redundancy: : [--------------------------------------->] 100 %



Loaded explosion and earthquake dataset:
Evenly balanced among classes in the train set.
As well as non train sets.
Distribution (Label: (counts, proportion)) of
Train ds:
earthquake: (4089, 0.4972)  |  explosion: (4135, 0.5028)  
Val ds:
earthquake: (613, 0.4972)  |  explosion: (620, 0.5028)  
Test ds:
earthquake: (406, 0.4939)  |  explosion: (416, 0.5061)  


In [3]:
use_time_augmentor = True
use_noise_augmentor = True
scaler_name = "normalize"
filter_name = None
band_min = 2
band_max = 4
highpass_freq = 0.075

use_tensorboard = False
use_liveplots = False
use_custom_callback = False
use_early_stopping = False
start_from_scratch = False
use_reduced_lr = False
log_data = False

load_test_set = False


ramLoader = RamLoader(loadData, 
                      handler, 
                      use_time_augmentor = use_time_augmentor, 
                      use_noise_augmentor = use_noise_augmentor, 
                      scaler_name = scaler_name,
                      filter_name = filter_name, 
                      band_min = band_min,
                      band_max = band_max,
                      highpass_freq = highpass_freq, 
                      load_test_set = load_test_set)
x_train, y_train, x_val, y_val, noiseAug = ramLoader.load_to_ram()


Fitting noise set time augmentor: [--------------------------------------->] 100 %

Fit process completed after 1.9875924587249756 seconds. Total datapoints fitted: 4239.
Average time per datapoint: 0.000468882391772818




Fitting train time augmentor: [--------------------------------------->] 100 %

Fit process completed after 3.7720634937286377 seconds. Total datapoints fitted: 8224.
Average time per datapoint: 0.0004586653080895717




Fitting validation time augmentor: [--------------------------------------->] 100 %

Fit process completed after 0.5587723255157471 seconds. Total datapoints fitted: 1233.
Average time per datapoint: 0.0004531811236948476




Stage one loading training set, timeAug: [--------------------------------------->] 100 %

Stage one loading validation set, timeAug: [--------------------------------------->] 100 %

Fit process of normalizer skipped as unecessary


Stage one loading noise set, timeAug: [--------------------------------------->] 100 %

Stage two loading noise set, labels and normalize scaler: [--------------------------------------->] 100 %

Fitting noise augmentor: [--------------------------------------->] 100 %



Stage two loading training set, labels and normalize scaler: [--------------------------------------->] 100 %

Stage two loading validation set, labels and normalize scaler: [--------------------------------------->] 100 %



Completed loading to RAM
Process took 0:00:19.242470 seconds.


In [4]:
val_gen = get_batch(num_workers = 1, 
                    max_queue_size = 1, 
                    traces = x_val, 
                    labels = y_val, 
                    batch_size = 16, 
                    noiseAug = noiseAug, 
                    num_channels = 3, 
                    use_time_augmentor = use_time_augmentor, 
                    norm_scale = True, 
                    shuffle = False)

In [5]:
x = next(val_gen)

In [7]:
x

In [ ]:
x.successful()

In [ ]:
del val_gen, x

In [78]:
def generator(x):
    while True:
        for i in range(len(x)):
            yield x[i]
enq.stop()

In [28]:
a = [i for i in range(10)]

enq = GeneratorEnqueuer(generator(a), use_multiprocessing = False)
enq.start(10, 1)
gen = enq.get()


In [118]:
def get_fancy(num_workers, max_queue_size, **kwargs):
    # Code similar to: https://github.com/watersink/Character-Segmentation/blob/master/data_generator.py
    try:
        enq = GeneratorEnqueuer(generator(**kwargs), use_multiprocessing = False)
        enq.start(workers = num_workers, max_queue_size = max_queue_size)
        gen_output = None
        while True:
            while enq.is_running():
                if not enq.queue.empty():
                    gen_output = enq.queue.get()
                    break
                else:
                    time.sleep(0.01)
            yield gen_output
            gen_output = None
    finally:
        if enq is not None:
            enq.stop()

In [119]:
gen = get_fancy(2, 10, x = a)

In [231]:
data = next(gen).get()
print(data)

8


In [137]:
data

4

In [126]:
x

0

In [111]:
import time

In [77]:
del gen